In [4]:
import tensorflow as tf
import numpy as np
import random
import csv
from collections import defaultdict

In [3]:
print("Reading data...")
import os   
userhome = os.path.expanduser('~')
csvfile= userhome + r'\Desktop\UCSD local\Web Mining\Assignment 2\flights.csv'
#dataFile = open(csvfile)
lines= []
with open(csvfile, "r") as f:
    cr = csv.DictReader(f, delimiter=',')
    for row in cr:
        lines.append(row)
#header = dataFile.readline()
#fields = ["constant"] + header.stridatp().replace('"','').split(';')
#featureNames = fields[:-1]
#labelName = fields[-1]
#lines = [[1.0] + [x for x in l.split(',')] for l in dataFile]
print("done")

Reading data...
done


In [5]:
### Randomizing samples
ind = [random.randint(0, len(lines)-1) for i in lines]
flights = [lines[ind[i]] for i in ind]

In [242]:
flightsSp = flights[:100000]
D = 60 # Delay threshold

In [243]:
i = 0
drop = [0]
for f in flightsSp:
    if f['ARRIVAL_DELAY'] == '':
        drop.append(i)
    i = i + 1
flightsSp = np.delete(flightsSp, drop, 0)

In [244]:
Airports = np.unique([x['ORIGIN_AIRPORT'] for x in flightsSp])
len(Airports)
Airlines = np.unique([x['AIRLINE'] for x in flightsSp])
len(Airlines)

14

In [245]:
Airlines

array(['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'MQ', 'NK', 'OO', 'UA',
       'US', 'VX', 'WN'], 
      dtype='<U2')

In [246]:
flightsTrain = flightsSp[:int(len(flightsSp)/2)]
flightsValid = flightsSp[int(len(flightsSp)/2):int(len(flightsSp)/2)+int(len(flightsSp)/4)]
flightsTest = flightsSp[int(len(flightsSp)/2)+int(len(flightsSp)/4):]

In [247]:
y_data = np.array([0.0 if float(f['ARRIVAL_DELAY']) < 0 else 1.0 if float(f['ARRIVAL_DELAY']) >= 0 and float(f['ARRIVAL_DELAY']) < D else 2.0 for f in flightsTrain])
y_valid = np.array([0.0 if float(f['ARRIVAL_DELAY']) < 0 else 1.0 if float(f['ARRIVAL_DELAY']) >= 0 and float(f['ARRIVAL_DELAY']) < D else 2.0 for f in flightsValid])
y_test = np.array([0.0 if float(f['ARRIVAL_DELAY']) < 0 else 1.0 if float(f['ARRIVAL_DELAY']) >= 0 and float(f['ARRIVAL_DELAY']) < D else 2.0 for f in flightsTest])

y_data.shape += (1, )
y_valid.shape += (1, )
y_test.shape += (1, )

globalAverage = float(sum(y_data) * 1.0 / len(y_data))

In [248]:
def feature(datum):
     # One feature dimension per month
    monthFeat = [0]*(12 - 1)
    if int(datum['MONTH']) != 12:
        monthFeat[int(datum['MONTH']) - 1] = 1.0
    
    f_airlines = [0]*len(Airlines)
    f_airlines[airlines_id[datum['AIRLINE']]] = 1    
    
    if  float(datum['DEPARTURE_DELAY']) < 0:
        delayFeat = 0
    elif  float(datum['DEPARTURE_DELAY']) >= 0 and  float(datum['DEPARTURE_DELAY']) < D:
        delayFeat = 1
    elif float(datum['DEPARTURE_DELAY']) >= D:
        delayFeat = 2
    
    feat = [1.0, delayFeat] + monthFeat + f_airlines
    return feat

In [249]:
airlines_id = dict(zip(Airlines,range(len(Airlines))))
x_data = np.matrix([feature(f) for f in flightsTrain])
x_valid = np.matrix([feature(f) for f in flightsValid])
x_test = np.matrix([feature(f) for f in flightsTest])

In [250]:
x_data[0]

matrix([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
          0.]])

In [251]:
flightsTrain[0]

{'AIRLINE': 'UA',
 'AIRLINE_DELAY': '28',
 'AIR_SYSTEM_DELAY': '0',
 'AIR_TIME': '149',
 'ARRIVAL_DELAY': '28',
 'ARRIVAL_TIME': '1532',
 'CANCELLATION_REASON': '',
 'CANCELLED': '0',
 'DAY': '18',
 'DAY_OF_WEEK': '4',
 'DEPARTURE_DELAY': '42',
 'DEPARTURE_TIME': '1248',
 'DESTINATION_AIRPORT': 'FLL',
 'DISTANCE': '1065',
 'DIVERTED': '0',
 'ELAPSED_TIME': '164',
 'FLIGHT_NUMBER': '1601',
 'LATE_AIRCRAFT_DELAY': '0',
 'MONTH': '6',
 'ORIGIN_AIRPORT': 'EWR',
 'SCHEDULED_ARRIVAL': '1504',
 'SCHEDULED_DEPARTURE': '1206',
 'SCHEDULED_TIME': '178',
 'SECURITY_DELAY': '0',
 'TAIL_NUMBER': 'N73256',
 'TAXI_IN': '3',
 'TAXI_OUT': '12',
 'WEATHER_DELAY': '0',
 'WHEELS_OFF': '1300',
 'WHEELS_ON': '1529',
 'YEAR': '2015'}

In [252]:
n, p = x_data.shape
k = 10

X = tf.placeholder('float', shape=[n,p])
# target vector
y = tf.placeholder('float', shape=[n,1])

# Create a new variable, and initialize to a vector of zeros
beta = tf.Variable(tf.zeros([p]))
#betaAp = tf.Variable(tf.constant([0.0]*len(flightsTrain), shape=[len(flightsTrain),1]))
alpha = tf.Variable(tf.constant([globalAverage]))
#theta = tf.Variable(tf.constant([0.0]*12, shape=[12,1]))

# interaction factors, randomly initialized
V = tf.Variable(tf.random_normal([k,p], stddev=0.01))

# estimate of y, initialized to 0
y_hat = tf.Variable(tf.zeros([n,1]))

In [253]:
linear_terms = tf.add(alpha, tf.reduce_sum(tf.multiply(beta,X),1, keep_dims=True))
interactions = (tf.multiply(0.5,tf.reduce_sum(tf.pow( tf.matmul(X, tf.transpose(V)), 2) - tf.matmul(tf.pow(X, 2), tf.transpose(tf.pow(V, 2))),1, keep_dims=True)))
y_hat = tf.add(linear_terms, interactions)

In [254]:
# L2 regularized sum of squares loss function over beta and V
lambda_b = tf.constant(0.001, name='lambda_b')
lambda_v = tf.constant(0.001, name='lambda_v')

l2_norm = (tf.reduce_sum(
            tf.add(
                tf.multiply(lambda_b, tf.pow(beta, 2)),
                tf.multiply(lambda_v, tf.pow(V, 2)))))

error = tf.reduce_mean(tf.square(y - y_hat))
loss = tf.add(error, l2_norm)

In [255]:
# Stochastic gradient descent
eta = tf.constant(0.1)
optimizer = tf.train.AdagradOptimizer(eta).minimize(loss)

In [256]:
# that's a lot of iterations
N_EPOCHS = 100
# Launch the graph.
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(N_EPOCHS):
        #indices = np.arange(n)
        #np.random.shuffle(indices)
        #x_data, y_data = x_data[indices], y_data[indices]
        sess.run(optimizer, feed_dict={X: x_data, y: y_data})

    print('MSE train: ', sess.run(error, feed_dict={X: x_data, y: y_data}))
    print('Loss (regularized error):', sess.run(loss, feed_dict={X: x_data, y: y_data}))
    print('Predictions:', sess.run(y_hat, feed_dict={X: x_data, y: y_data}))
    predictions_train = sess.run(y_hat, feed_dict={X: x_data, y: y_data})
    print('Learnt weights:', sess.run(beta, feed_dict={X: x_data, y: y_data}))
    print('Learnt factors:', sess.run(V, feed_dict={X: x_data, y: y_data}))
    
    linear_terms = tf.add(alpha, tf.reduce_sum(tf.multiply(beta,x_valid),1, keep_dims=True))
    interactions = (tf.multiply(0.5,tf.reduce_sum(tf.pow( tf.matmul(tf.cast(x_valid, tf.float32), tf.transpose(V)), 2) - tf.matmul(tf.cast(tf.pow(x_valid, 2), tf.float32), tf.transpose(tf.pow(V, 2))),1, keep_dims=True)))
    predictions_valid = tf.add(linear_terms, interactions)
    predictions_valid = [np.round(x) for x in predictions_valid.eval()]
    
    linear_terms = tf.add(alpha, tf.reduce_sum(tf.multiply(beta,x_test),1, keep_dims=True))
    interactions = (tf.multiply(0.5,tf.reduce_sum(tf.pow( tf.matmul(tf.cast(x_test, tf.float32), tf.transpose(V)), 2) - tf.matmul(tf.cast(tf.pow(x_test, 2), tf.float32), tf.transpose(tf.pow(V, 2))),1, keep_dims=True)))
    predictions_test = tf.add(linear_terms, interactions)
    predictions_test = [np.round(x) for x in predictions_test.eval()]
    

MSE train:  0.203391
Loss (regularized error): 0.207553
Predictions: [[ 0.71606487]
 [ 0.18984789]
 [ 0.82571912]
 ..., 
 [ 0.04661187]
 [ 0.73812878]
 [ 0.69418246]]
Learnt weights: [-0.10597961  0.59478092  0.0192516   0.03382099 -0.00996979 -0.00270214
 -0.00577645  0.00856004 -0.0006979  -0.02035226 -0.03296826 -0.03086338
 -0.02586823 -0.02300419 -0.00932483 -0.00610413 -0.0898926   0.04021576
  0.04133448  0.02897581  0.00367897  0.04151224  0.03674183 -0.10402381
  0.0130885   0.00788708 -0.08777658]
Learnt factors: [[  2.68682260e-02   3.23547348e-02   1.19934725e-02  -5.88242011e-03
   -1.05991326e-02   5.79003245e-04   5.97141264e-03   1.31906718e-02
   -7.36789429e-04  -2.82591325e-03  -1.78468018e-03  -8.90343916e-03
    4.98220790e-03   1.16501423e-02  -4.96737985e-03   1.21251866e-03
    1.18406340e-02   9.30488389e-03   9.28193424e-03  -2.01758090e-03
    8.76545091e-04   2.54426561e-02   1.78961689e-03   5.15784929e-03
    5.97201427e-03   1.65909790e-02  -3.85740609e-0

In [257]:
correct_valid = [(a==b) for (a,b) in zip(predictions_valid,y_valid)]
acc_valid = sum(correct_valid) * 1.0 / len(correct_valid)
acc_valid

array([ 0.71090917])

In [258]:
correct_train = [(a==b) for (a,b) in zip(predictions_train,y_data)]
acc_train = sum(correct_train) * 1.0 / len(correct_train)
acc_train
correct_test = [(a==b) for (a,b) in zip(predictions_test,y_test)]
acc_test = sum(correct_test) * 1.0 / len(correct_test)
acc_test

array([ 0.71398884])

In [259]:
bc = sum([y_test[i]==0 and predictions_test[i]!=0 for i in range(len(y_test))])
abc = sum([y_test[i]==0 for i in range(len(y_test))])
df = sum([y_test[i]==1 and predictions_test[i]!=1 for i in range(len(y_test))])
ndef = sum([y_test[i]==1 for i in range(len(y_test))])
gh = sum([y_test[i]==2 and predictions_test[i]!=2 for i in range(len(y_test))])
ghi = sum([y_test[i]==2 for i in range(len(y_test))])
tri_ber = (bc*1.0/abc+df*1.0/ndef+gh*1.0/ghi)*1.0/3
print(tri_ber)

[ 0.44887377]
